In [1]:
from mainnet_launch.constants import *
from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks
import pandas as pd
from mainnet_launch.data_fetching.get_events import fetch_events, get_each_event_in_contract
import plotly.express as px
from mainnet_launch.abis import BALANCER_INCENTIVES_GAUGE_ABI, CURVE_INCENTIVES_GAUGE_ABI, ERC_20_ABI

pool = "0x88794C65550DeB6b4087B7552eCf295113794410"
incentives_gauge = "0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd"  # balancer

contract = ETH_CHAIN.client.eth.contract(address=incentives_gauge, abi=CURVE_INCENTIVES_GAUGE_ABI)
# events = get_each_event_in_contract(contract, ETH_CHAIN, start_block=ETH_CHAIN.block_autopool_first_deployed, end_block=ETH_CHAIN.get_block_near_top())

In [2]:
dai = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
dai_contract = ETH_CHAIN.client.eth.contract(address=dai, abi=ERC_20_ABI)
dai_to_incentives_gauge = fetch_events(
    dai_contract.events.Transfer,
    ETH_CHAIN,
    start_block=ETH_CHAIN.block_autopool_first_deployed,
    end_block=ETH_CHAIN.get_block_near_top(),
    argument_filters={"to": incentives_gauge},
)

Fetched 11 logs for <class 'web3._utils.datatypes.Transfer'> from 20,722,908 to 23,319,652 (2,596,745 blocks)


In [8]:
from mainnet_launch.data_fetching.get_state_by_block import identity_with_bool_success
from multicall import Call


address = "0x570eA5C8A528E3495EE9883910012BeD598E8814"

# Create a multicall Call object for rewardPerTokenStored
reward_per_token_call = Call(
    target=address,
    function="rewardPerTokenStored()(uint256)",
    returns=[("balGauge_rewardPerTokenStored", identity_with_bool_success)],
)

# reward per token stored has not changed durunin gthis period

df = get_raw_state_by_blocks(
    [reward_per_token_call], dai_to_incentives_gauge["block"], ETH_CHAIN, include_block_number=True
)
dai_to_incentives_gauge = pd.merge(dai_to_incentives_gauge, df, left_on="block", right_on="block", how="left")
dai_to_incentives_gauge["dai_amount"] = dai_to_incentives_gauge["value"].apply(lambda x: x / 10**18)
dai_to_incentives_gauge.index = df.index

In [9]:
dai_to_incentives_gauge

,event,block,transactionIndex,log_index,hash,from,to,value,balGauge_rewardPerTokenStored_x,dai_amount,delay,balGauge_rewardPerTokenStored_y
timestamp,,,,,,,,,,,,
2025-06-19 13:00:59+00:00,Transfer,22738686,15,58,0xd2c0bcb5cd36c677eb519c5384811fe882d51bfd741f...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,7000000000000000000000,54672987900060315648,7000.0000,NaN,54672987900060315648
2025-06-27 11:09:47+00:00,Transfer,22795386,47,265,0xdd2f399212e38bbab7c34802b63d13f94895fc4c1b16...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,7000000000000000000000,54672987900060315648,7000.0000,1.0,54672987900060315648
2025-07-03 04:17:59+00:00,Transfer,22836262,338,567,0xe4723a36d4aac4ae6db3757ed02a84466214ea7e5132...,0x6c214A193F921E9b7D52aA7387Fa8882Ee50DA0f,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,9100000000000000,54672987900060315648,0.0091,1.0,54672987900060315648
2025-07-04 21:15:11+00:00,Transfer,22848477,69,200,0x25a1a5e209035e564b991d63aa133c254c7cc69df5c4...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,7000000000000000000000,54672987900060315648,7000.0000,1.0,54672987900060315648
2025-07-14 13:41:59+00:00,Transfer,22917781,108,386,0xa6171e003066230d007083289b9bf0ff3fc13a4293bc...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,7000000000000000000000,54672987900060315648,7000.0000,1.0,54672987900060315648
2025-07-21 10:29:47+00:00,Transfer,22966977,89,368,0x65619f609671d23de9d568f868f40e68386111cb8ce2...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,7000000000000000000000,54672987900060315648,7000.0000,1.0,54672987900060315648
2025-07-28 20:05:11+00:00,Transfer,23019908,331,713,0x9892b6123f329c5d8dd9dc6736bc8ce23066b8ac42dc...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,7000000000000000000000,54672987900060315648,7000.0000,1.0,54672987900060315648
2025-08-05 21:56:59+00:00,Transfer,23077694,123,525,0xc576f6228fabf6da5549cd77d4ed11867cb243e42900...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,5000000000000000000000,54672987900060315648,5000.0000,1.0,54672987900060315648
2025-08-12 15:05:11+00:00,Transfer,23125744,125,425,0xe53494e83ace608890d73598eae7acfcd61154512789...,0x7f6494D4fBEA1c06daC2250A3FCa81003bF8D20C,0xdee29b7b6dF3576280bFBbC088ee9eBAA767C0Bd,5000000000000000000000,54672987900060315648,5000.0000,1.0,54672987900060315648


In [18]:
dai_to_incentives_gauge["gap_between_reloading_dai"] = dai_to_incentives_gauge["gap_between_reloading_dai"].clip(
    lower=pd.Timedelta(0)
)
dai_to_incentives_gauge[["gap_between_reloading_dai", "dai_amount"]]

,gap_between_reloading_dai,dai_amount
timestamp,,
2025-06-19 13:00:59+00:00,NaT,7000.0000
2025-06-27 11:09:47+00:00,0 days 22:08:48,7000.0000
2025-07-03 04:17:59+00:00,0 days 00:00:00,0.0091
2025-07-04 21:15:11+00:00,0 days 00:00:00,7000.0000
2025-07-14 13:41:59+00:00,2 days 16:26:48,7000.0000
2025-07-21 10:29:47+00:00,0 days 00:00:00,7000.0000
2025-07-28 20:05:11+00:00,0 days 09:35:24,7000.0000
2025-08-05 21:56:59+00:00,1 days 01:51:48,5000.0000
2025-08-12 15:05:11+00:00,0 days 00:00:00,5000.0000


In [19]:
dai_to_incentives_gauge[["gap_between_reloading_dai", "dai_amount"]].sum()

gap_between_reloading_dai    8 days 16:40:12
dai_amount                        61760.0091
dtype: object

In [20]:
dai_to_incentives_gauge.index.min() - dai_to_incentives_gauge.index.max()

Timedelta('-72 days +07:18:24')

In [ ]:
px.scatter(dai_to_incentives_gauge, x="delay", y="dai_amount", title="DAI sent to Incentives Gauge over time")

In [ ]:
for k, v in events.items():
    print(f"Event: {k}, Count: {len(v)}")

In [ ]:
events["Deposit"]